# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

## Загрузка и подготовка данных

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from numpy.random import RandomState

from sklearn.metrics import mean_squared_error

from sklearn.linear_model import LinearRegression

In [2]:
data_0=pd.read_csv('/datasets/geo_data_0.csv')
data_1=pd.read_csv('/datasets/geo_data_1.csv')
data_2=pd.read_csv('/datasets/geo_data_2.csv')

In [3]:
display(data_0.head())
data_0.info()
data_0.describe()

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.500419,0.250143,2.502647,92.500000
std,0.871832,0.504433,3.248248,44.288691
min,-1.408605,-0.848218,-12.088328,0.000000
25%,-0.072580,-0.200881,0.287748,56.497507
50%,0.502360,0.250252,2.515969,91.849972
75%,1.073581,0.700646,4.715088,128.564089
max,2.362331,1.343769,16.003790,185.364347


In [4]:
display(data_1.head())
data_1.info()
data_1.describe()

,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,1.141296,-4.796579,2.494541,68.825000
std,8.965932,5.119872,1.703572,45.944423
min,-31.609576,-26.358598,-0.018144,0.000000
25%,-6.298551,-8.267985,1.000021,26.953261
50%,1.153055,-4.813172,2.011479,57.085625
75%,8.621015,-1.332816,3.999904,107.813044
max,29.421755,18.734063,5.019721,137.945408


In [5]:
display(data_2.head())
data_2.info()
data_2.describe()

,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.002023,-0.002081,2.495128,95.000000
std,1.732045,1.730417,3.473445,44.749921
min,-8.760004,-7.084020,-11.970335,0.000000
25%,-1.162288,-1.174820,0.130359,59.450441
50%,0.009424,-0.009482,2.484236,94.925613
75%,1.158535,1.163678,4.858794,130.595027
max,7.238262,7.844801,16.739402,190.029838


Выбросов и некоректных значений не замечено. 

In [6]:
data_0[data_0.id.duplicated(keep=False)].sort_values(by='id')

,id,f0,f1,f2,product
66136,74z30,1.084962,-0.312358,6.990771,127.643327
64022,74z30,0.741456,0.459229,5.153109,140.771492
51970,A5aEY,-0.180335,0.935548,-2.094773,33.020205
3389,A5aEY,-0.039949,0.156872,0.209861,89.249364
69163,AGS9W,-0.933795,0.116194,-3.655896,19.230453
42529,AGS9W,1.454747,-0.479651,0.683380,126.370504
931,HZww2,0.755284,0.368511,1.863211,30.681774
7530,HZww2,1.061194,-0.373969,10.430210,158.828695
63593,QcMuo,0.635635,-0.473422,0.862670,64.578675
1949,QcMuo,0.506563,-0.323775,-2.215583,75.496502


In [7]:
data_1[data_1.id.duplicated(keep=False)].sort_values(by='id')

,id,f0,f1,f2,product
5849,5ltQ6,-3.435401,-12.296043,1.999796,57.085625
84461,5ltQ6,18.213839,2.191999,3.993869,107.813044
1305,LHZR0,11.170835,-1.945066,3.002872,80.859783
41906,LHZR0,-8.989672,-4.286607,2.009139,57.085625
2721,bfPNe,-9.494442,-5.463692,4.006042,110.992147
82178,bfPNe,-6.202799,-4.820045,2.995107,84.038886
47591,wt4Uk,-9.091098,-8.109279,-0.002314,3.179103
82873,wt4Uk,10.259972,-9.376355,4.994297,134.766305


In [8]:
data_2[data_2.id.duplicated(keep=False)].sort_values(by='id')

,id,f0,f1,f2,product
45404,KUPhW,0.231846,-1.698941,4.990775,11.716299
55967,KUPhW,1.211150,3.176408,5.543540,132.831802
11449,VF7Jo,2.122656,-0.858275,5.746001,181.716817
49564,VF7Jo,-0.883115,0.560537,0.723601,136.233420
44378,Vcm5J,-1.229484,-2.439204,1.222909,137.968290
95090,Vcm5J,2.587702,1.986875,2.482245,92.327572
28039,xCHr8,1.633027,0.368135,-2.378367,6.120525
43233,xCHr8,-0.847066,2.101796,5.597130,184.388641


<b>Вывод по импорту и преданализу данных</b> В данных отсутствуют пропуски и выбросы. Есть дубликаты но они связаны в одним идентификатором, мб разные варианты освоения одной и той же скважины. 

## Обучение и проверка модели

In [9]:
features_0=data_0.drop(['product','id'],axis=1)
target_0=data_0['product']
features_1=data_1.drop(['product','id'],axis=1)
target_1=data_1['product']
features_2=data_2.drop(['product','id'],axis=1)
target_2=data_2['product']

In [10]:
features_0_train, features_0_valid, target_0_train, target_0_valid = train_test_split(features_0,target_0,test_size=0.25,random_state=12345)
features_1_train, features_1_valid, target_1_train, target_1_valid = train_test_split(features_1,target_1,test_size=0.25,random_state=12345)
features_2_train, features_2_valid, target_2_train, target_2_valid = train_test_split(features_2,target_2,test_size=0.25,random_state=12345)

In [11]:
model = LinearRegression()

model.fit(features_0_train, target_0_train)
predictions_0 = pd.Series(model.predict(features_0_valid))
model.fit(features_1_train, target_1_train)
predictions_1 = pd.Series(model.predict(features_1_valid))
model.fit(features_2_train, target_2_train)
predictions_2 = pd.Series(model.predict(features_2_valid))

In [12]:
print('RMSE  региона 0 составили - {} при среднем значении предсказанных запасов - {}, среднем целевого признака - {}'.format(round(mean_squared_error(target_0_valid, predictions_0)**0.5,2),round(predictions_0.mean(),2),round(target_0_valid.mean(),2)))
print('RMSE  региона 1 составили - {} при среднем значении предсказанных запасов - {}, среднем целевого признака - {}'.format(round(mean_squared_error(target_1_valid, predictions_1)**0.5,2),round(predictions_1.mean(),2),round(target_1_valid.mean(),2)))
print('RMSE  региона 2 составили - {} при среднем значении предсказанных запасов - {}, среднем целевого признака - {}'.format(round(mean_squared_error(target_2_valid, predictions_2)**0.5,2),round(predictions_2.mean(),2),round(target_2_valid.mean(),2)))

RMSE  региона 0 составили - 37.58 при среднем значении предсказанных запасов - 92.59, среднем целевого признака - 92.08
RMSE  региона 1 составили - 0.89 при среднем значении предсказанных запасов - 68.73, среднем целевого признака - 68.72
RMSE  региона 2 составили - 40.03 при среднем значении предсказанных запасов - 94.97, среднем целевого признака - 94.88


На модели 1 линейная регрессия показала высокое кач-во прогнозирования. По оставшимся - высокий уровень ошибки. Применение ЛР возможно. 

## Подготовка к расчёту прибыли

In [13]:
product_margin = 450_000
one_well_cost = 50_000_000
profitable_volume = round(one_well_cost/product_margin,2)
print('Объем добычи, обеспечивающий безубыточность - {} тыс.баррелей'.format(profitable_volume) )

Объем добычи, обеспечивающий безубыточность - 111.11 тыс.баррелей


In [14]:
metrics_df = pd.DataFrame({'участок':[0,1,2]}).set_index('участок')
metrics_df['средние_запасы'] = [np.mean(target_0), np.mean(target_1), np.mean(target_2)]
metrics_df['отношение_среднего_к_требуемому'] = np.round(metrics_df['средние_запасы']/profitable_volume,2)
metrics_df['количество_скважин_с_достаточными_запасами'] = [target_0[target_0>profitable_volume].count(),target_1[target_1>profitable_volume].count(), target_2[target_2>profitable_volume].count()]
metrics_df['доля_скважин_с_достаточным_запасами'] = metrics_df['количество_скважин_с_достаточными_запасами']/100000
metrics_df.T

участок,0,1,2
средние_запасы,92.50000,68.82500,95.00000
отношение_среднего_к_требуемому,0.83000,0.62000,0.86000
количество_скважин_с_достаточными_запасами,36584.00000,16537.00000,38178.00000
доля_скважин_с_достаточным_запасами,0.36584,0.16537,0.38178


Вывод по подготовке к расчету прибыли.
Объем добычи, обеспечивающий безубыточность, существенно выше средних значений запасов по каждому из регионов.

## Расчёт прибыли и рисков 

In [15]:
def profit_assume(target_place,predictions_sample):
    predictions_200 = predictions_sample.sort_values(ascending=False).head(200)
    target_200 = target_place[predictions_200.index]
    if len(target_200)!=len(predictions_200): print('target 200 <> predictions 200 =>',len(target_200),len(predictions_200))
    assume = sum(target_200)*product_margin-one_well_cost*200
    return assume

In [16]:
state = RandomState(12345)
def bootstrap_place(target_place,predictions_place):
    profits = []
    for i in range(1,1000):
        predictions_sample = predictions_place.sample(n=500, replace=True, random_state=state)
        profits.append(profit_assume(target_place,predictions_sample))
    profits=pd.Series(profits)
    mean_profit = round(np.mean(profits)/1000000000,2)
    loss_rate = len(profits[profits<0])/1000
    upper = round(profits.quantile(0.975)/1000000000,2)
    lower = round(profits.quantile(0.025)/1000000000,2)
    return mean_profit, loss_rate, upper, lower

In [17]:
mean_profit, loss_rate, upper, lower = bootstrap_place(target_0_valid.reset_index(drop=True),predictions_0)
print('Значения рассчетов для участка 0:')
print('Средняя прибыль проекта {} млрд.руб.'.format(round(mean_profit,2)))
print('Вероятность получения убытка - {}%'.format(round(loss_rate*100,2)))
print('95-% доверительный интервал для участка 0 в диапазоне от {} до {} млрд.руб.'.format(lower,upper))

Значения рассчетов для участка 0:
Средняя прибыль проекта 0.4 млрд.руб.
Вероятность получения убытка - 6.9%
95-% доверительный интервал для участка 0 в диапазоне от -0.11 до 0.91 млрд.руб.


In [18]:
mean_profit, loss_rate, upper, lower = bootstrap_place(target_1_valid.reset_index(drop=True),predictions_1)
print('Значения рассчетов для участка 1:')
print('Средняя прибыль проекта {} млрд.руб.'.format(round(mean_profit,2)))
print('Вероятность получения убытка - {}%'.format(round(loss_rate*100,2)))
print('95-% доверительный интервал для участка 0 в диапазоне от {} до {} млрд.руб.'.format(lower,upper))

Значения рассчетов для участка 1:
Средняя прибыль проекта 0.46 млрд.руб.
Вероятность получения убытка - 0.7%
95-% доверительный интервал для участка 0 в диапазоне от 0.08 до 0.86 млрд.руб.


In [19]:
mean_profit, loss_rate, upper, lower = bootstrap_place(target_2_valid.reset_index(drop=True),predictions_2)
print('Значения рассчетов для участка 2:')
print('Средняя прибыль проекта {} млрд.руб.'.format(round(mean_profit,2)))
print('Вероятность получения убытка - {}%'.format(round(loss_rate*100,2)))
print('95-% доверительный интервал для участка 0 в диапазоне от {} до {} млрд.руб.'.format(lower,upper))

Значения рассчетов для участка 2:
Средняя прибыль проекта 0.39 млрд.руб.
Вероятность получения убытка - 6.4%
95-% доверительный интервал для участка 0 в диапазоне от -0.11 до 0.93 млрд.руб.


## Вывод по результатам расчетов

По результатам выполнения расчетов ботстрап максимальная средняя прибыль у первого участка 0,46 млрд.руб. и минимальная вероятнасть получения убытка 0.7%. Предлагаем выбрать первый участок исходя из полученных выше данных.  